### Build dataframe that contains the number of people (based on their WiFi device) that visited every pair of stores in our store list

In [10]:
import pandas as pd                                                                                                                                 
from pandas.tools.plotting import parallel_coordinates
# Numpy
import numpy as np

# SciPy
from scipy import constants as phys_consts
# Think this one is from scipy
from skimage import io as skimg_io

# Matplotlib
import matplotlib.pyplot as plt
import matplotlib.dates as mplDates
from matplotlib.path import Path as mplPath
import matplotlib.patches as mplPatches
from matplotlib.gridspec import GridSpec
font = {'weight' : 'bold',
        'size'   : 22}
plt.rc('font', **font)

%matplotlib inline


Define polygons for the various regions in pixels, then convert to x-y

In [11]:
import matplotlib.path as mplPath

kruidvat = [[1468,996],[1588,996],[1652,934],[1652,824],[1468,824]]
albertheijn = [[1783,1364],[2149,1364],[2220,1291],[2220,1001],[1783,1001],[1783,1049],[1718,1089],[1718,1172],[1754,1172],[1754,1291]]
hema = [[1656,743],[1848,743],[1848,551],[1656,551]]
abnamro = [[288,1372],[662,1372],[662,1290],[587,1290],[587,1146],[344,1146],[344,1185],[300,1218],[300,1288],[288,1288]]
hagenouw = [[2002,925],[2221,925],[2221,799],[2002,799]]
taesoen = [[2006,709],[2150,709],[2150,785],[2207,785],[2207,639],[2150,639],[2150,593],[2095,593],[2095,568],[2006,569]]
gamemania = [[1308,997],[1423,997],[1423,774],[1308,774]]

def convertToXY(arr):
    for i in range(0,len(arr)):
        arr[i][0] = (arr[i][0]/10.14) - 24.0
        arr[i][1] = (arr[i][1]/11.3) - 73.98
    return arr

KruidPath = mplPath.Path(np.array(convertToXY(kruidvat)))
ABNPath = mplPath.Path(np.array(convertToXY(abnamro)))
AHPath = mplPath.Path(np.array(convertToXY(albertheijn)))
HemaPath = mplPath.Path(np.array(convertToXY(hema)))

Load the WiFi data with min signal strength at sensor of -75 dBm

In [12]:
pDate = '2015-12-25'
appId = 'cb004'
dfFitAH = pd.read_csv('data/outputPdFileFitter{0}_{1}.csv'.format(appId,pDate),sep=',')

dfFitAH = dfFitAH[(dfFitAH['localMac'] == 0) & (dfFitAH['maxValue'] > -75)].copy()            
dfFitAH.head()

volumes = {}
volumes['InKruidvat'] = KruidPath
volumes['InAH'] = AHPath

dfFitAH['InKruidvat'] = dfFitAH.apply(lambda x: int(KruidPath.contains_point((x['x'],x['y']))),axis=1)
dfFitAH['ABNAmro'] = dfFitAH.apply(lambda x: int(ABNPath.contains_point((x['x'],x['y']))),axis=1)
dfFitAH['InAH'] = dfFitAH.apply(lambda x: int(AHPath.contains_point((x['x'],x['y']))),axis=1)
dfFitAH['InHema'] = dfFitAH.apply(lambda x: int(HemaPath.contains_point((x['x'],x['y']))),axis=1)

# Check that data is in the form we want
dfFitAH[dfFitAH['InKruidvat'] > 0].head()


,Unnamed: 0,sourceMac,measurementTimestamp,x,y,maxValue,maxProbe,localMac,InKruidvat,ABNAmro,InAH,InHema
118,118,bd81542f-1e52-4ee0-8024-3cf03011e740,1451008204767,137.326444,6.341843,-74.000000,123,0,1,0,0,0
168,168,bd81542f-1e52-4ee0-8024-3cf03011e740,1451018583677,134.815370,3.096709,-74.500000,123,0,1,0,0,0
499,499,bd81542f-1e52-4ee0-8024-3cf03011e740,1451034894689,134.541808,3.979136,-74.000000,123,0,1,0,0,0
1466,1466,f4c1306d-96ed-470c-b3f5-3bbb53670b0a,1451039672446,126.186915,14.146507,-52.000000,123,0,1,0,0,0
1732,1732,064d0577-1387-4f26-b512-b8c826cfbd5d,1451040529421,134.670098,11.587335,-63.666667,123,0,1,0,0,0


Group by device (source mac address) and get the max of it being in any of the stores

In [13]:
dfFitG = dfFitAH.groupby('sourceMac').agg({'InKruidvat' : np.max})
dfFitG['ABNAmro'] = dfFitAH.groupby('sourceMac')['ABNAmro'].max()
dfFitG['InAH'] = dfFitAH.groupby('sourceMac')['InAH'].max()
dfFitG['InHema'] = dfFitAH.groupby('sourceMac')['InHema'].max()

### Determine the number of devices that were in each store

In [14]:
kr_count = dfFitG[dfFitG['InKruidvat'] > 0].count()[0]
ah_count = dfFitG[dfFitG['InAH'] > 0].count()[0]
hm_count = dfFitG[dfFitG['InHema'] > 0].count()[0]
ab_count = dfFitG[dfFitG['ABNAmro'] > 0].count()[0]

Make new dataframe with columns being all combinations of 2 stores,  sum up the column values for each combination

In [15]:
from  itertools import combinations
store_combs = list(combinations(dfFitG.columns,2))
df = pd.concat([dfFitG[comb[1]].add(dfFitG[comb[0]]) for comb in store_combs], axis=1, keys=store_combs)
df.columns = df.columns.map('-'.join)

Determine the number for each combination based on the sum of the column values being > 1, this means the device was in both locations. Build matrix of visit combinations where m$_{ij}$ corresponds to number that was both in store i and j

In [16]:
visit_index = ['InKruidvat', 'ABNAmro', 'InAH', 'InHema']
visit_array = [[0]*len(visit_index) for i in range(len(visit_index))]
for cols in df.columns:
    comb_count = df[df[cols] > 1].count()[0]
    piv = cols.index('-')
    vis_num0 = visit_index.index(cols[0:piv])
    vis_num1 = visit_index.index(cols[piv+1:])
    visit_array[vis_num0][vis_num1] = comb_count
    # this is so the matrix is symmetric
    visit_array[vis_num1][vis_num0] = comb_count
 
# fill in the diagnol elements
visit_array[0][0] = kr_count
visit_array[1][1] = ab_count
visit_array[2][2] = ah_count
visit_array[3][3] = hm_count
visit_array

[[30, 7, 14, 6], [7, 174, 10, 7], [14, 10, 50, 9], [6, 7, 9, 26]]

### Finally convert the matrix to dataframe and display

In [17]:
dfFinal = pd.DataFrame(visit_array, columns=visit_index)

In [18]:
dfFinal.index = visit_index
dfFinal

,InKruidvat,ABNAmro,InAH,InHema
InKruidvat,30,7,14,6
ABNAmro,7,174,10,7
InAH,14,10,50,9
InHema,6,7,9,26
